In [4]:
#Import Statements

import numpy as np


In [ ]:
def forward_pass():
    pass

In [ ]:
def backward_pass():
    pass

In [ ]:
#Activation Function
def ReLU(z):
    return np.maximum(z,0,z)

In [11]:
def sigmoid(z):
    return 1/(np.exp(-1*z) + 1)

In [9]:
class Network:
    def __init__(self,layers):
        if len(layers)<2:
            print("Cannot create Neural Network with less than 2 layers")
            return
        self.layers = layers
        self.weights = []
        self.biases = []
        
        for i in range(1, len(layers)):
            rows = layers[i-1] #784
            cols = layers[i] #28
            layer_weight = np.random.random((rows,cols))
            layer_bias = np.random.random(layers[i])
            self.weights.append(layer_weight)
            self.biases.append(layer_bias)